In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # This is the analysis pipeline for the calculation of tau and branching factor of neural data from the hippocampus. 
# 
# The data are take from: https://datadryad.org/stash/dataset/doi:10.7272/Q61N7ZC3

# ### Importing the branching factor/tau estimator
# 
# For documentation regards the estimator see https://mrestimator.readthedocs.io/en/latest/doc/gettingstarted.html, Spitzner, F. P., Dehning, J., Wilting, J., Hagemann, A., P. Neto, J., Zierenberg, J., & Priesemann, V. (2021).

# ### Importing relevant non-standard modules
# All of which can be found on github https://github.com/JanBellingrath/Hippocampal_Neuron_Data_Processing


import pipeline.initiation_functions as init
from collections import namedtuple


# ### Defining each animal via its short name and its directory
Animal = namedtuple('Animal', {'short_name', 'directory'})

#'con': Animal('con','/home/bellijjy/Conley.tar/Conley'),
#'Cor': Animal('Cor','/home/bellijjy/Corriander.tar/Corriander'),
#'cha': Animal('cha','/home/bellijjy/Chapati.tar/Chapati/Chapati'),
#'dav': Animal('dav','/home/bellijjy/Dave.tar/Dave/Dave'),
#'dud': Animal('dud','/home/bellijjy/Dudley'),
animals_dict = {'fra' : Animal('fra', '/local2/Jan/Frank/Frank'),
           'gov' : Animal('gov', '/local2/Jan/Government/Government'),
           'egy' : Animal('egy', '/local2/Jan/Egypt/Egypt'), 
           'remy': Animal('remy', '/local2/Jan/Remy/Remy'),
           "Fiv" : Animal("Fiv", "/home/dekorvyb/Downloads/Fiv"),
           "bon" : Animal("bon", "/home/dekorvyb/Downloads/Bon")}




# animals to analyze
animal_list = [ "bon"] #'egy', 'con', 'dav', 'fra', 'gov', 'remy', 'bon', 'dud', 'Cor', "Fiv"

# areas to analyze
area_list = ['CA1'] #CA3
len_time_chunk = 90

init.intrinsic_time_scale_estimation_for_all_animals(animal_list,
                                                     area_list,
                                                     len_time_chunk,
                                                     animals_dict,
                                                     target_directory = "/home/dekorvyb/Documents/gov_CA1_CA3/",
                                                     mode = "debug",
                                                     check_duplicates = True)



#dav ca3 done
#cor ca1 done
#rest not yet

